# Understanding Ontologies with Cognee

This notebook demonstrates how to work with ontologies in scientific research using the Cognee framework. We'll explore how ontologies can enhance our understanding and querying of scientific papers.

## What is an Ontology?

An ontology is a formal representation of knowledge that defines:
- Concepts within a domain
- Relationships between concepts
- Properties and attributes
- Rules and constraints

Key terms:
- **Classes**: Categories or types (e.g., Disease, Symptom)
- **Instances**: Specific examples of classes (e.g., Type 2 Diabetes)
- **Properties**: Relationships between classes/instances (e.g., hasSymptom)
- **Axioms**: Logical statements defining relationships

## Setup

First, let's install the required packages and set up our environment:

In [17]:
# Install required package
# !pip install cognee

In [18]:
# Import required libraries
import cognee
import asyncio
from cognee.shared.logging_utils import get_logger
import os
import textwrap
from cognee.api.v1.search import SearchType
from cognee.api.v1.visualize.visualize import visualize_graph

logger = get_logger()

# Set up OpenAI API key (required for Cognee's LLM functionality)
os.environ["LLM_API_KEY"] = "your-api-key-here"  # Replace with your API key

## Creating the Pipeline

Let's create a pipeline that will:
1. Clean existing data
2. Process scientific papers
3. Apply ontological knowledge

In [19]:
async def run_pipeline(ontology_path=None):
    # Clean existing data
    await cognee.prune.prune_data()
    await cognee.prune.prune_system(metadata=True)
    
    # Set up path to scientific papers
    scientific_papers_dir = os.path.join(
        os.path.dirname(os.path.dirname(os.path.abspath("."))), 
        "cognee",
        "examples",
        "data", 
        "scientific_papers/"
    )
    
    # Add papers to the system
    await cognee.add(scientific_papers_dir)
    
    # Cognify with optional ontology
    return await cognee.cognify(ontology_file_path=ontology_path)

async def query_pipeline(questions):
    answers = []
    for question in questions:
        search_results = await cognee.search(
            query_type=SearchType.GRAPH_COMPLETION,
            query_text=question,
        )
        answers.append(search_results)
    return answers

## Running the Demo

Let's test our system with some medical questions, comparing results with and without ontological knowledge:

In [7]:
# Test questions
questions = [
    "What are common risk factors for Type 2 Diabetes?",
    "What preventive measures reduce the risk of Hypertension?",
    "What symptoms indicate possible Cardiovascular Disease?",
    "What diseases are associated with Obesity?"
]

# Path to medical ontology
ontology_path = "examples/python/ontology_input_example/enriched_medical_ontology_with_classes.owl"  # Update with your ontology path

# Run with ontology
print("\n--- Results WITH ontology ---\n")
await run_pipeline(ontology_path=ontology_path)
answers_with = await query_pipeline(questions)
for q, a in zip(questions, answers_with):
    print(f"Q: {q}\nA: {a}\n")


--- Results WITH ontology ---


2025-03-26T15:05:17.231943Z [warning  ] File /Users/vasilije/cognee/cognee/.cognee_system/databases/cognee_graph.pkl not found. Initializing an empty graph. [cognee.shared.logging_utils]
2025-03-26T15:05:17.234283Z [info     ] Graph deleted successfully.    [cognee.shared.logging_utils]
2025-03-26T15:05:17.237309Z [info     ] Database deleted successfully. [cognee.shared.logging_utils]User d3a8d4b0-7559-4a30-89fa-5f8ccec78676 has registered.

2025-03-26T15:05:17.305948Z [info     ] Pipeline run started: `4b84e400-23fc-5976-bbb4-f8ee303eed81` [run_tasks(tasks: [Task], data)]
2025-03-26T15:05:17.306299Z [info     ] Coroutine task started: `resolve_data_directories` [run_tasks(tasks: [Task], data)]
2025-03-26T15:05:17.306931Z [info     ] Coroutine task started: `ingest_data` [run_tasks(tasks: [Task], data)]
2025-03-26T15:05:17.800072Z [info     ] Coroutine task completed: `ingest_data` [run_tasks(tasks: [Task], data)]
2025-03-26T15:05:17.800400Z [info     

/Users/vasilije/cognee/.venv/lib/python3.11/site-packages/dlt/destinations/impl/sqlalchemy/merge_job.py:194: SAWarning: Table 'file_metadata' already exists within the given MetaData - not copying.
  staging_table_obj = table_obj.to_metadata(
/Users/vasilije/cognee/.venv/lib/python3.11/site-packages/dlt/destinations/impl/sqlalchemy/merge_job.py:229: SAWarning: implicitly coercing SELECT object to scalar subquery; please use the .scalar_subquery() method to produce a scalar subquery.
  order_by=order_dir_func(order_by_col),



2025-03-26T15:05:18.168160Z [info     ] Coroutine task started: `extract_graph_from_data` [run_tasks(tasks: [Task], data)]

16:05:18 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai



LiteLLM completion() model= gpt-4o-mini; provider = openai

...



LiteLLM completion() model= gpt-4o-mini; provider = openai

16:07:15 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai



LiteLLM completion() model= gpt-4o-mini; provider = openai

16:07:21 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai



LiteLLM completion() model= gpt-4o-mini; provider = openaiQ: What are common risk factors for Type 2 Diabetes?
A: ['Common risk factors for Type 2 Diabetes that can be inferred from the context include:\n- Hypertension\n- Heart failure\n- Cholesterol issues\n- Poor cardiovascular health\nThese factors indicate a connection between lifestyle and the development of Type 2 Diabetes.']

Q: What preventive measures reduce the risk of Hypertension?
A: ['Preventive measures that reduce the risk of hypertension include moderate coffee consumption. Studies indicate that drinking 1 to 3 cups of coffee per day is associated with a decreased risk of developing hypertension, particularly in individuals who have never smoked and are rapid caffeine metabolizers.']

Q: What symptoms indicate possible Cardiovascular Disease?
A: ['Symptoms that may indicate possible Cardiovascular Disease (CVD) include:\n- Hypertension (high blood pressure)\n- Diabetes\n- High cholesterol\n- Physical inactivity\n- Inc

In [8]:
# Run without ontology
print("\n--- Results WITHOUT ontology ---\n")
await run_pipeline()
answers_without = await query_pipeline(questions)
for q, a in zip(questions, answers_without):
    print(f"Q: {q}\nA: {a}\n")


--- Results WITHOUT ontology ---


2025-03-26T15:09:44.143281Z [info     ] Graph deleted successfully.    [cognee.shared.logging_utils]
2025-03-26T15:09:44.167825Z [info     ] Database deleted successfully. [cognee.shared.logging_utils]User cfb3d7df-13ec-4404-bdd9-da9def0c3ee5 has registered.

2025-03-26T15:09:44.234228Z [info     ] Pipeline run started: `4b84e400-23fc-5976-bbb4-f8ee303eed81` [run_tasks(tasks: [Task], data)]
2025-03-26T15:09:44.234622Z [info     ] Coroutine task started: `resolve_data_directories` [run_tasks(tasks: [Task], data)]
2025-03-26T15:09:44.235313Z [info     ] Coroutine task started: `ingest_data` [run_tasks(tasks: [Task], data)]
2025-03-26T15:09:44.483281Z [info     ] Coroutine task completed: `ingest_data` [run_tasks(tasks: [Task], data)]
2025-03-26T15:09:44.483708Z [info     ] Coroutine task completed: `resolve_data_directories` [run_tasks(tasks: [Task], data)]
2025-03-26T15:09:44.484101Z [info     ] Pipeline run completed: `4b84e400-23fc-5976-bbb4-f8ee303

/Users/vasilije/cognee/.venv/lib/python3.11/site-packages/dlt/destinations/impl/sqlalchemy/merge_job.py:194: SAWarning: Table 'file_metadata' already exists within the given MetaData - not copying.
  staging_table_obj = table_obj.to_metadata(
/Users/vasilije/cognee/.venv/lib/python3.11/site-packages/dlt/destinations/impl/sqlalchemy/merge_job.py:229: SAWarning: implicitly coercing SELECT object to scalar subquery; please use the .scalar_subquery() method to produce a scalar subquery.
  order_by=order_dir_func(order_by_col),



2025-03-26T15:09:44.849823Z [info     ] Coroutine task started: `extract_graph_from_data` [run_tasks(tasks: [Task], data)]

16:09:44 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai



LiteLLM completion() model= gpt-4o-mini; provider = openai

...



LiteLLM completion() model= gpt-4o-mini; provider = openai

16:10:30 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai



LiteLLM completion() model= gpt-4o-mini; provider = openai

16:10:34 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai



LiteLLM completion() model= gpt-4o-mini; provider = openaiQ: What are common risk factors for Type 2 Diabetes?
A: ['Common risk factors for Type 2 Diabetes include: hypertension, cardiovascular disease (CVD), and being aged 20 years and above.']

Q: What preventive measures reduce the risk of Hypertension?
A: ['Preventive measures to reduce the risk of hypertension include moderate coffee consumption, as studies have shown that it is associated with a decreased risk of developing hypertension. Additionally, lifestyle factors such as maintaining a healthy weight, reducing sodium intake, and regular physical activity may also contribute to lower hypertension risk.']

Q: What symptoms indicate possible Cardiovascular Disease?
A: ['Symptoms indicating possible cardiovascular disease include hypertension, heart failure, and atrial fibrillation. Additionally, heavy coffee consumption has been linked to increased risks of coronary heart disease, while moderate coffee consumption may decreas

## Visualizing the Knowledge Graph

Let's visualize how our ontology connects different medical concepts:

In [16]:
# Generate and display the visualization
html = await visualize_graph(destination_file_path= "/Users/vasilije/cognee/.artifacts/graph.html")
from cognee.api.v1.visualize.start_visualization_server import start_visualization_server

start_visualization_server(port=8003)



2025-03-26T15:18:43.650136Z [info     ] Graph visualization saved as /Users/vasilije/cognee/.artifacts/graph.html [cognee.shared.logging_utils]
2025-03-26T15:18:43.650674Z [info     ] The HTML file has been stored at path: /Users/vasilije/cognee/.artifacts/graph.html [cognee.shared.logging_utils]

OSError: [Errno 48] Address already in use

## Understanding the Results

The demonstration above shows how ontologies enhance our analysis by:

1. **Making Connections**: 
   - Linking related medical concepts even when not explicitly stated
   - Identifying relationships between symptoms, diseases, and risk factors

2. **Standardizing Terms**: 
   - Unifying different ways of referring to the same medical condition
   - Ensuring consistent terminology across documents

3. **Enabling Inference**: 
   - Drawing conclusions based on ontological relationships
   - Discovering implicit connections in the data

## Next Steps

To learn more about Cognee and ontologies:
1. Check out the [Cognee documentation](https://docs.cognee.ai/)
2. Explore more examples in the `examples` directory
3. Try creating your own domain-specific ontology

Remember to:
- Place your scientific papers in the appropriate directory
- Update the ontology path to point to your .owl file
- Replace the API key with your own OpenAI key